In [1]:
# get the preprocessed data from the preprocess file
from preprocess import *

## Model Building

In [ ]:
# interpolation + add-k smoothing
def probability_interpolation():
    return

In [ ]:
# build a model using interpolation
def language_model_interpolation():
    return

## Model Evaluation

In [ ]:
def perplexity_interpolation():
    return

## Text Generation